In [1]:
import pandas as pd
import numpy as np
import math
import yaml

In [16]:
def weighted_position_investing(input_dataframe, params):
    """
    Args:
        input_dataframe: pandas.DataFrame
                    Column position [0] needs to be close price.
                    All other columns need to consist of "Buy"/"Sell"/"Hold" decisions.
        investment_capital: Float
                    The amount of "money" able to be invested.
        buy_weight: Float
                    Values must be between 0-1. Represents the amount to invest from
                    available remaining capital (in procentage). Ie, 0.5 -> 50% of remaining
                    available capital on buy.
        sell_weight: Float
                    Values must be between 0-1. Represents the amount of invested capital
                    to sell out of (in procentage). Ie, 0.5 -> 50% of invested capital on sell.
    Returns:
        pandas.DataFrame
            Profit calculations appended to input_dataframe.
    """
    
    # DECONSTRUCT PARAMS
    investment_capital = params['capital']
    buy_weight = params['buy']
    sell_weight = params['sell']
    
    output_dataframe = pd.DataFrame(index=input_dataframe.index, columns=input_dataframe.columns)
    
    # LOOP THROUGH COLUMNS
    for col in input_dataframe.columns[:]:
        if col == 'close':
            del output_dataframe['close']
        else:
            current_profit = 0
            holding_value = 0
            resulting_col = col

            wallet = investment_capital
            positions_close = []
            positions_amount = []
            investment_amount = round((wallet * buy_weight), 2)

            # LOOP THROUGH EACH ROW
            for i in range(len(input_dataframe)):

                current_close_price = input_dataframe["close"].iloc[i]
                label = input_dataframe[col].iloc[i]

                # ON BUY = 0
                if((wallet > 0) & (label == 0)):
                    # check to see if invest wallet amount or investment amount.
                    if wallet < investment_amount:
                        amount = calculate_amount(current_close_price, wallet)
                        positions_close.append(current_close_price)
                        positions_amount.append(amount)
                        # remove money from wallet
                        wallet -= wallet
                        investment_amount = round((wallet * buy_weight), 2)

                    else:
                        amount = calculate_amount(current_close_price, investment_amount)
                        positions_close.append(current_close_price)
                        positions_amount.append(amount)
                        # remove money from wallet
                        wallet -= investment_amount
                        investment_amount = round((wallet * buy_weight), 2)

                    # ADD NEW PROFIT TO DATAFRAME
                    output_dataframe[resulting_col].iloc[i] = round(current_profit, 2)


                # ON SELL = 1
                elif((len(positions_close) > 0) & (label == 1)):
                    n = get_weighted_list_length(positions_close, sell_weight)
                    for close_p, amount_p in zip(positions_close[:n], positions_amount[:n]):
                        # get sell price and compare to buy price
                        selling_price = current_close_price * amount_p
                        buying_price = round((close_p * amount_p),2)
                        # update wallet to have money again, i.e selling_price
                        wallet += round(selling_price, 2)
                        # register profit/loss
                        profit = calculate_profit(buying_price, selling_price)
                        current_profit += round(profit, 2)
                    investment_amount = round((wallet * buy_weight), 2)

                    # ADD NEW PROFIT TO DATAFRAME
                    output_dataframe[resulting_col].iloc[i] = round(current_profit, 2)

                    # EMPTY POSITIONS LISTS
                    positions_close = positions_close[n:]
                    positions_amount = positions_amount[n:]

                # ON HOLD = 2
                else:
                    # Since we are holding, do nothing except udpate current profit
                    # ADD NEW PROFIT TO DATAFRAME
                    output_dataframe[resulting_col].iloc[i] = round(current_profit, 2)

            # SELL OFF OUT REMAINING POSITIONS
            current_close_price = input_dataframe["close"].iloc[-1]
            for close_p, amount_p in zip(positions_close, positions_amount):
                selling_price = current_close_price * amount_p
                buying_price = close_p * amount_p
                profit = calculate_profit(buying_price, selling_price)
                current_profit += round(profit, 2)
            
            output_dataframe[resulting_col].iloc[-1] = round(current_profit, 2)
    
    return output_dataframe

In [11]:
def calculate_amount(unit_price, budget):
    amount = budget / unit_price
    return amount

In [4]:
def calculate_profit(buy_price, sell_price):
    profit = sell_price - buy_price
    return profit

In [13]:
def get_weighted_list_length(list_item, weight):
    list_length = int(math.ceil(len(list_item) * weight))
    return list_length

In [6]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [7]:
def process(src):
    return pd.read_csv(src, delimiter=',',index_col=0)

In [8]:
def profiles_calculation(yaml_configs):
    configs = load_yaml(yaml_configs)
    profiles = configs['investing_profiles']
    data = process(configs['data'])
    investment_capital = configs['investment_capital']
    profiles
    results = []
    for profile in profiles:
        buy_weight = profiles[profile]['buy_weight']
        sell_weight = profiles[profile]['sell_weight']
        result = multi_position_investing(data, investment_capital,buy_weight, sell_weight)
        results.append(result)
    return results